In [28]:
import os
import csv
import pandas as pd
import sqlite3

In [36]:
current_database_folder = "C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24"

In [38]:
current_database_path = "C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24/current_database.db"

In [39]:
def get_database_connection(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    return conn, cursor

In [40]:
current_database_conn, current_database_cursor = get_database_connection(current_database_path)

In [42]:
current_database_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables_current = [row[0] for row in current_database_cursor.fetchall()]

In [43]:
tables_current

['EastDVD_20240726_RM_ARC',
 'EastDVD_20240726_RM_ATL',
 'EastDVD_20240726_RM_CEN',
 'EastDVD_20240726_V_ATL_A',
 'EastDVD_20240726_V_ATL_B',
 'EastDVD_20240726_V_ATL_C',
 'EastDVD_20240726_V_CEN_A',
 'EastDVD_20240726_V_CEN_B',
 'EastDVD_20240726_V_CEN_C',
 'EastDVD_20240726_V_NOR_A',
 'WestDVD_20240726_RM_ARC',
 'WestDVD_20240726_RM_PAC',
 'WestDVD_20240726_V_NOR_A',
 'WestDVD_20240726_V_PAC_A',
 'WestDVD_20240726_V_PAC_B']